In [1]:
!pip install keras_tuner -q

In [2]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Flatten
from kerastuner.tuners import RandomSearch

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build a deep learning model with Batch Normalization
def build_mnist_model(optimizer='adam'):
    model = Sequential([
        Flatten(input_shape=(28, 28)),  # Flatten the input
        Dense(128),
        BatchNormalization(),  # Batch Normalization layer
        tf.keras.layers.ReLU(),  # ReLU activation function
        Dense(64),
        BatchNormalization(),
        tf.keras.layers.ReLU(),
        Dense(10, activation='softmax')  # Output layer with 10 units for 10 digits
    ])
    
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Use Keras Tuner to search for the best hyperparameters
def build_tuner_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(Dense(units=hp_units, activation='relu'))

    # Tune the number of hidden layers and units in each hidden layer
    hp_hidden_layers = hp.Int('hidden_layers', min_value=1, max_value=3)
    for _ in range(hp_hidden_layers):
        model.add(Dense(units=hp_units, activation='relu'))

    model.add(Dense(10, activation='softmax'))  # Output layer with 10 units for 10 digits

    # Compile the model
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Instantiate the tuner and perform the search
tuner = RandomSearch(
    build_tuner_model,
    objective='val_accuracy',
    max_trials=5,  # You can increase this for a more thorough search
    directory='keras_tuner',
    project_name='mnist_tuning'
)

# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Print the summary of the best model
best_model.summary()


Trial 3 Complete [00h 01m 44s]
val_accuracy: 0.9815000295639038

Best val_accuracy So Far: 0.9815000295639038
Total elapsed time: 00h 05m 49s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
256               |384               |units
2                 |2                 |hidden_layers
sgd               |rmsprop           |optimizer



KeyboardInterrupt: 